In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
##!pip install torch==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu121

## Imports

In [3]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import gdown
import torch
import sys
from tqdm import tqdm
import shutil
import os.path as osp
import glob
import cv2
import numpy as np
import torch.nn.functional as F
from math import log10
from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
import torch.optim as optim


## Getting Ground truth and noisy image pairs

In [4]:
class ImagePairs(Dataset):
    def __init__(self, noisy_dir, gt_dir, transform=None):
        self.noisy_files=sorted(os.listdir(noisy_dir))
        self.gt_files=sorted(os.listdir(gt_dir))
        self.noisy_dir=noisy_dir
        self.gt_dir=gt_dir
        self.transform=transform
    def __len__(self):
        return len(self.noisy_files)

    def __getitem__(self, idx):
        noisy = Image.open(os.path.join(self.noisy_dir, self.noisy_files[idx])).convert("RGB")
        gt = Image.open(os.path.join(self.gt_dir, self.gt_files[idx])).convert("RGB")
        if self.transform:
            noisy = self.transform(noisy)
            gt = self.transform(gt)
        return noisy, gt

transform=T.Compose([T.ToTensor()])

train_dataset=ImagePairs("/kaggle/input/dlp-may-2025-nppe-3/archive/train/train", "/kaggle/input/dlp-may-2025-nppe-3/archive/train/gt", transform )
train_loader=DataLoader(train_dataset, batch_size=1, shuffle=True)


## Getting Validation Image and GT Pairs

In [5]:
val_dataset=ImagePairs("/kaggle/input/dlp-may-2025-nppe-3/archive/val/val", "/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt", transform)
val_loader=DataLoader(val_dataset, batch_size=1, shuffle=True)

## Cloning Github directories for models

In [6]:
!git clone https://github.com/swz30/MPRNet.git
!git clone https://github.com/xinntao/ESRGAN.git


Cloning into 'MPRNet'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 339 (delta 65), reused 56 (delta 56), pack-reused 259 (from 1)
Receiving objects: 100% (339/339), 92.17 KiB | 3.29 MiB/s, done.
Resolving deltas: 100% (187/187), done.
Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 225 (delta 43), reused 41 (delta 41), pack-reused 176 (from 1)
Receiving objects: 100% (225/225), 24.86 MiB | 42.07 MiB/s, done.
Resolving deltas: 100% (86/86), done.


## Getting Pretrained model weights for MPRNet

In [7]:
os.makedirs("/kaggle/working/MPRNet/pretrained_models", exist_ok=True)

url = "https://drive.google.com/uc?id=1LODPt9kYmxwU98g96UrRA0_Eh5HYcsRw"
output = "/kaggle/working/MPRNet/pretrained_models/model_denoising.pth"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1LODPt9kYmxwU98g96UrRA0_Eh5HYcsRw
To: /kaggle/working/MPRNet/pretrained_models/model_denoising.pth
100%|██████████| 63.0M/63.0M [00:00<00:00, 124MB/s]


'/kaggle/working/MPRNet/pretrained_models/model_denoising.pth'

## Finetuning MPRNet using training data

In [8]:
sys.path.append("/kaggle/working/MPRNet/Denoising")

from MPRNet import MPRNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
denoise_model = MPRNet().to(device)

checkpoint = torch.load(
    "/kaggle/working/MPRNet/pretrained_models/model_denoising.pth",
    map_location=device
)
denoise_model.load_state_dict(checkpoint['state_dict'])


<All keys matched successfully>

In [9]:
# Charbonnier Loss
class CharbonnierLoss(nn.Module):
    def __init__(self, eps=1e-3):
        super(CharbonnierLoss, self).__init__()
        self.eps = eps
    def forward(self, x, y):
        diff = x - y
        loss = torch.mean(torch.sqrt(diff * diff + self.eps * self.eps))
        return loss

criterion = CharbonnierLoss().to(device)
optimizer = optim.Adam(denoise_model.parameters(), lr=1e-5) 


In [10]:
scaler = GradScaler() 

num_epochs = 2

for epoch in range(num_epochs):
    denoise_model.train()
    train_loss = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for noisy, clean in pbar:
        noisy, clean = noisy.to(device), clean.to(device)
    
        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=(device.type=='cuda')):
            restored = denoise_model(noisy) 
            
            loss = 0
            for stage_output in restored:
                if stage_output.shape != clean.shape:
                    stage_output = F.interpolate(stage_output, size=clean.shape[2:], mode='bilinear', align_corners=False)
                loss += criterion(stage_output, clean)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        pbar.set_postfix(loss=f"{train_loss / (pbar.n or 1):.4f}")

    
    denoise_model.eval()
    val_loss = 0
    with torch.no_grad():
        for noisy, clean in val_loader:
            noisy, clean = noisy.to(device), clean.to(device)
            restored = denoise_model(noisy)
            for stage_output in restored:
                if stage_output.shape != clean.shape:
                    stage_output = F.interpolate(stage_output, size=clean.shape[2:], mode='bilinear', align_corners=False)
                val_loss += criterion(stage_output, clean).item()
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}")

    torch.cuda.empty_cache()


torch.save(denoise_model.state_dict(), "mprnet_finetuned.pth")
print("Fine-tuned model saved.")


/tmp/ipykernel_19/2832299922.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/2:   0%|          | 0/1105 [00:00<?, ?it/s]/tmp/ipykernel_19/2832299922.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=='cuda')):
Epoch 1/2: 100%|██████████| 1105/1105 [06:19<00:00,  2.92it/s, loss=0.0310]


Validation Loss: 0.0227


Epoch 2/2: 100%|██████████| 1105/1105 [06:02<00:00,  3.05it/s, loss=0.0295]


Validation Loss: 0.0223
Fine-tuned model saved.


In [11]:
import torch
torch.cuda.empty_cache()

## Code for denoising data

In [12]:
transform = T.Compose([T.ToTensor()])
to_pil = T.ToPILImage()

def denoise_image(img_path):
    noisy = Image.open(img_path).convert("RGB")
    noisy_tensor = transform(noisy).unsqueeze(0).to(device)

    with torch.no_grad():
        restored = denoise_model(noisy_tensor)[0]
    restored = torch.clamp(restored, 0, 1)
    restored_img = to_pil(restored.squeeze().cpu())
    return restored_img


def denoise_folder(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    image_files = sorted(os.listdir(input_dir))
    for file in tqdm(image_files, desc="Denoising"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)
        try:
            denoised_img = denoise_image(input_path)
            denoised_img.save(output_path)
        except Exception as e:
            print(f"Skipping {file}: {e}")



## Denoising train data

In [13]:
input_dir = "/kaggle/input/dlp-may-2025-nppe-3/archive/train/train"
output_dir = "/kaggle/working/denoised_results_train"

denoise_folder(input_dir, output_dir)

Denoising: 100%|██████████| 1105/1105 [03:28<00:00,  5.31it/s]


## Denoising validation data

In [14]:
input_dir = "/kaggle/input/dlp-may-2025-nppe-3/archive/val/val"
output_dir = "/kaggle/working/denoised_results_val"

denoise_folder(input_dir, output_dir)

Denoising: 100%|██████████| 267/267 [00:50<00:00,  5.33it/s]


## Loading Pretrained weights of ESRGAN model

In [15]:
os.makedirs("/kaggle/working/ESRGAN/models", exist_ok=True)

In [16]:
!gdown --folder https://drive.google.com/drive/folders/17VYV_SoZZesU6mbxz2dMAIccSSlqLecY -O /kaggle/working/ESRGAN/models/

Retrieving folder contents
Processing file 1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb- RRDB_ESRGAN_x4_old_arch.pth
Processing file 1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene RRDB_ESRGAN_x4.pth
Processing file 1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr RRDB_PSNR_x4_old_arch.pth
Processing file 1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN RRDB_PSNR_x4.pth
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-
To: /kaggle/working/ESRGAN/models/ESRGAN_models/RRDB_ESRGAN_x4_old_arch.pth
100%|███████████████████████████████████████| 67.0M/67.0M [00:00<00:00, 145MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene
To: /kaggle/working/ESRGAN/models/ESRGAN_models/RRDB_ESRGAN_x4.pth
100%|███████████████████████████████████████| 66.9M/66.9M [00:00<00:00, 153MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr
To: /kaggle/wor

In [17]:
!rm /kaggle/working/ESRGAN/LR/baboon.png /kaggle/working/ESRGAN/LR/comic.png

In [18]:
# src_folder = "/kaggle/working/denoised_results"
# dst_folder = "/kaggle/working/ESRGAN/LR"

# os.makedirs(dst_folder, exist_ok=True)

# for file_name in os.listdir(src_folder):
#     if file_name.lower().endswith((".png", ".jpg", ".jpeg")):
#         shutil.copy(os.path.join(src_folder, file_name),
#                     os.path.join(dst_folder, file_name))


## Getting a training loader with denoised images

In [19]:
transform=T.Compose([T.ToTensor()])

next_train=ImagePairs("/kaggle/working/denoised_results_train", "/kaggle/input/dlp-may-2025-nppe-3/archive/train/gt", transform )
dl_sr=DataLoader(next_train, batch_size=1, shuffle=True,num_workers=4)


In [20]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()


## Finetune RRDBNet

In [21]:
sys.path.append("/kaggle/working/ESRGAN")
import RRDBNet_arch as arch

device = torch.device('cuda')
model_path = '/kaggle/working/ESRGAN/models/ESRGAN_models/RRDB_ESRGAN_x4.pth'
sr_net = arch.RRDBNet(3, 3, 64, 23, gc=32).to(device)

checkpoint = torch.load(model_path, map_location=device)
sr_net.load_state_dict(checkpoint, strict=True)
print(f"Loaded pretrained ESRGAN checkpoint from {model_path}")

sr_net.train()
optimizer = torch.optim.Adam(sr_net.parameters(), lr=1e-4)
scaler_sr = GradScaler()
crit_L1 = torch.nn.L1Loss()
SR_EPOCHS = 2

for ep in range(1, SR_EPOCHS+1):
    running = 0.0
    pbar = tqdm(dl_sr, desc=f"SR Fine-tune Epoch {ep}/{SR_EPOCHS}")
    for lr_noisy_t, hr_t in pbar:
        lr_noisy_t, hr_t = lr_noisy_t.to(device), hr_t.to(device)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device.type=='cuda')):
            sr = sr_net(lr_noisy_t)
            loss = crit_L1(sr, hr_t)

        scaler_sr.scale(loss).backward()
        scaler_sr.step(optimizer)
        scaler_sr.update()

        running += loss.item()
        pbar.set_postfix(loss=f"{running / (pbar.n or 1):.4f}")

    torch.cuda.empty_cache()

FT_SR = os.path.join("/kaggle/working", 'rrdbnet_finetuned.pth')
torch.save(sr_net.state_dict(), FT_SR)
print(f"Fine-tuned ESRGAN model saved at {FT_SR}")

/tmp/ipykernel_19/2768310278.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_sr = GradScaler()


Loaded pretrained ESRGAN checkpoint from /kaggle/working/ESRGAN/models/ESRGAN_models/RRDB_ESRGAN_x4.pth


SR Fine-tune Epoch 1/2:   0%|          | 0/1105 [00:00<?, ?it/s]/tmp/ipykernel_19/2768310278.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type=='cuda')):
SR Fine-tune Epoch 2/2: 100%|██████████| 1105/1105 [12:37<00:00,  1.46it/s, loss=0.0090]


Fine-tuned ESRGAN model saved at /kaggle/working/rrdbnet_finetuned.pth


## Super Resolution on Val data

In [22]:
os.makedirs("/kaggle/working/final_results_val",exist_ok=True)
test_img_folder = '/kaggle/working/denoised_results_val/*'


idx = 0
for path in sorted(glob.glob(test_img_folder)):
    ##print(path)
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = sr_net(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('/kaggle/working/final_results_val/{:s}.png'.format(base), output)


1 val_00001


[ WARN:0@2660.030] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


2 val_00002
3 val_00003
4 val_00004
5 val_00005
6 val_00006
7 val_00007
8 val_00008
9 val_00009
10 val_00010
11 val_00011
12 val_00012
13 val_00013
14 val_00014
15 val_00015
16 val_00016
17 val_00017
18 val_00018
19 val_00019
20 val_00020
21 val_00021
22 val_00022
23 val_00023
24 val_00024
25 val_00025
26 val_00026
27 val_00027
28 val_00028
29 val_00029
30 val_00030
31 val_00031
32 val_00032
33 val_00033
34 val_00034
35 val_00035
36 val_00036
37 val_00037
38 val_00038
39 val_00039
40 val_00040
41 val_00041
42 val_00042
43 val_00043
44 val_00044
45 val_00045
46 val_00046
47 val_00047
48 val_00048
49 val_00049
50 val_00050
51 val_00051
52 val_00052
53 val_00053
54 val_00054
55 val_00055
56 val_00056
57 val_00057
58 val_00058
59 val_00059
60 val_00060
61 val_00061
62 val_00062
63 val_00063
64 val_00064
65 val_00065
66 val_00066
67 val_00067
68 val_00068
69 val_00069
70 val_00070
71 val_00071
72 val_00072
73 val_00073
74 val_00074
75 val_00075
76 val_00076
77 val_00077
78 val_00078
79 val_

## Code for checking PSNR

In [23]:
transform = T.ToTensor()

def calculate_psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
    if mse == 0:
        return 100
    return 10 * log10(1.0 / mse.item())

def evaluate_psnr(input_dir, gt_dir):
    noisy_files = sorted(os.listdir(input_dir))
    gt_files = sorted(os.listdir(gt_dir))

    psnr_scores = []

    for noisy_file, gt_file in tqdm(zip(noisy_files, gt_files), total=len(noisy_files), desc="Evaluating PSNR"):
        denoised = Image.open(os.path.join(input_dir, noisy_file)).convert("RGB")
        gt = Image.open(os.path.join(gt_dir, gt_file)).convert("RGB")
        
        denoised_tensor = transform(denoised).unsqueeze(0)
        gt_tensor = transform(gt).unsqueeze(0)
        if denoised_tensor.shape != gt_tensor.shape:
            denoised_tensor = F.interpolate(denoised_tensor, size=gt_tensor.shape[2:], mode="bilinear", align_corners=False)
    
        psnr = calculate_psnr(denoised_tensor, gt_tensor)
        psnr_scores.append(psnr)

    avg_psnr = sum(psnr_scores) / len(psnr_scores)
    return avg_psnr, psnr_scores


denoised_dir = "/kaggle/working/final_results_val"
gt_dir = "/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt"

avg_psnr, all_psnrs = evaluate_psnr(denoised_dir, gt_dir)

print(f"Average PSNR: {avg_psnr:.2f} dB")


Evaluating PSNR: 100%|██████████| 267/267 [00:10<00:00, 25.90it/s]

Average PSNR: 39.29 dB


## Making Inference on Test Data

In [24]:
class TestDataset(Dataset):
    def __init__(self, noisy_dir, transform=None):
        self.noisy_files=sorted(os.listdir(noisy_dir))
        self.noisy_dir=noisy_dir
        self.transform=transform
    def __len__(self):
        return len(self.noisy_files)

    def __getitem__(self, idx):
        noisy = Image.open(os.path.join(self.noisy_dir, self.noisy_files[idx])).convert("RGB")
        if self.transform:
            noisy = self.transform(noisy)
        return noisy

transform=T.Compose([T.ToTensor()])

test_dataset=TestDataset("/kaggle/input/dlp-may-2025-nppe-3/archive/test", transform )
test_loader=DataLoader(test_dataset, batch_size=4, shuffle=True)

## Denoising Test Images

In [25]:
input_dir = "/kaggle/input/dlp-may-2025-nppe-3/archive/test"
output_dir = "/kaggle/working/denoised_results_test"

denoise_folder(input_dir, output_dir)

Denoising: 100%|██████████| 60/60 [00:11<00:00,  5.21it/s]


## Increasing Resolution

In [26]:
os.makedirs('/kaggle/working/final_results_test', exist_ok=True)


In [27]:
test_img_folder = '/kaggle/working/denoised_results_test/*'


idx = 0
for path in sorted(glob.glob(test_img_folder)):
    ##print(path)
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = sr_net(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('/kaggle/working/final_results_test/{:s}.png'.format(base), output)


1 test_00001
2 test_00002
3 test_00003
4 test_00004
5 test_00005
6 test_00006
7 test_00007
8 test_00008
9 test_00009
10 test_00010
11 test_00011
12 test_00012
13 test_00013
14 test_00014
15 test_00015
16 test_00016
17 test_00017
18 test_00018
19 test_00019
20 test_00020
21 test_00021
22 test_00022
23 test_00023
24 test_00024
25 test_00025
26 test_00026
27 test_00027
28 test_00028
29 test_00029
30 test_00030
31 test_00031
32 test_00032
33 test_00033
34 test_00034
35 test_00035
36 test_00036
37 test_00037
38 test_00038
39 test_00039
40 test_00040
41 test_00041
42 test_00042
43 test_00043
44 test_00044
45 test_00045
46 test_00046
47 test_00047
48 test_00048
49 test_00049
50 test_00050
51 test_00051
52 test_00052
53 test_00053
54 test_00054
55 test_00055
56 test_00056
57 test_00057
58 test_00058
59 test_00059
60 test_00060


In [28]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in sorted(os.listdir(folder_path)):
        print(filename)
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            print(image_id)
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/final_results_test'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)


test_00001.png
gt_00001
test_00002.png
gt_00002
test_00003.png
gt_00003
test_00004.png
gt_00004
test_00005.png
gt_00005
test_00006.png
gt_00006
test_00007.png
gt_00007
test_00008.png
gt_00008
test_00009.png
gt_00009
test_00010.png
gt_00010
test_00011.png
gt_00011
test_00012.png
gt_00012
test_00013.png
gt_00013
test_00014.png
gt_00014
test_00015.png
gt_00015
test_00016.png
gt_00016
test_00017.png
gt_00017
test_00018.png
gt_00018
test_00019.png
gt_00019
test_00020.png
gt_00020
test_00021.png
gt_00021
test_00022.png
gt_00022
test_00023.png
gt_00023
test_00024.png
gt_00024
test_00025.png
gt_00025
test_00026.png
gt_00026
test_00027.png
gt_00027
test_00028.png
gt_00028
test_00029.png
gt_00029
test_00030.png
gt_00030
test_00031.png
gt_00031
test_00032.png
gt_00032
test_00033.png
gt_00033
test_00034.png
gt_00034
test_00035.png
gt_00035
test_00036.png
gt_00036
test_00037.png
gt_00037
test_00038.png
gt_00038
test_00039.png
gt_00039
test_00040.png
gt_00040
test_00041.png
gt_00041
test_00042.png
g